# Itertools Walkthrough
----
This notebook is a walkthrough of the project based tutorial on the itertools Python module.  The tutorial is found [here](https://realpython.com/python-itertools/).

In [36]:
from math import sqrt
import itertools as it
import timeit
import pprint
PP = pprint.PrettyPrinter(indent=4)

## Groupers are tasty fish
----

In [4]:

# Example of low level iterator generators/operators
print(list(zip([1, 2, 3],['a', 'b', 'c'])))
# Zip iterates through each iterator and returns combined tuples of
# ith element in each iteratable object

print(list(map(len, ['abcd', 'ef', 'ghit'])))
# The map function calls the iter() function on an iterable and
# applies the function passed to the value returned by the
# next() function
print(list(map(lambda x: x**2, [2,4,6])))

# Since iterators are iterable, you can combine these functions
print(list(map(sum, zip([1,2,3], [5,6,7]))))

def hypotenuese(args):
    return sqrt(args[0]**2 + args[1]**2)
# Testing out map with multiple inputs on a custom function
# -- TURNS OUT YOU CAN ONLY PASS A SINGLE ARGUMENT, MAKE IT COUNT
print(list(map(hypotenuese, zip([1,2,3],[4,5,6]))))

[(1, 'a'), (2, 'b'), (3, 'c')]
[4, 2, 4]
[4, 16, 36]
[6, 8, 10]
[4.123105625617661, 5.385164807134504, 6.708203932499369]


In [15]:
# Iterators use lazy execution which can drastically improve 
# memory usage and processing speed

# Example - bad option first
def naive_grouper(inputs, n):
    n_groups = len(inputs) // n
    return [tuple(inputs[i*n:(i+1)*n] for i in range(n_groups))]

# This works fine for small lists (to return tuples) but if you 
# pass a list of 100 million numbers it will consume roughly
# 4.5GB of ram and take 11 seconds.

# Let's implement a better solution  - This executes 3-5x as fast and, 
# due to the use of iter(), consumes ~ 630x less RAM
def better_grouper(inputs, n):
    iters = [iter(inputs)]*n
    return zip(*iters)

In [11]:
# Timing the execution of this code.  This is going to take a while.
# Since timeit will execute this code 1M times, we'll keep this short
TEST_CODE = """
for _ in naive_grouper(range(1000), 10):
  pass
"""
print(timeit.timeit(stmt=TEST_CODE, globals=globals()))

25.550513171001512


In [17]:
# Timing the execution of our improved method
TEST_CODE = """
for _ in better_grouper(range(1000), 10):
    pass
"""
print(timeit.timeit(stmt=TEST_CODE, globals=globals()))

9.42678318400067


In [23]:
print(naive_grouper([1,2,3,4,5,6, 7,8,9,10],2))
print(list(better_grouper([1,2,3,4,5,6, 7, 8, 9, 10], 2)))

[([1, 2], [3, 4], [5, 6], [7, 8], [9, 10])]
[(1, 2), (3, 4), (5, 6), (7, 8), (9, 10)]


In [31]:
# better_grouper could still be ...well, better.
# It doesn't had cases where the number of elements is not
# a multiple of the grouping number, e.g.
print("Better Grouper Output, still lacking:\n",
      list(better_grouper([1,2,3,4,5,6, 7, 8, 9, 10], 3)))

# let's see if we can improve on that. We can use the itertools function
# zip_longest().  This will take any number of iterables PLUS a fill
# value that defaults to None
def betterest_grouper(inputs, n, fillvalue=None):
    iters = [iter(inputs)]*n
    return it.zip_longest(*iters, fillvalue=fillvalue)

# Now let's try that again
print("Betterest Grouper can handle this:\n",
      list(betterest_grouper([1,2,3,4,5,6, 7, 8, 9, 10], 3)))

Better Grouper Output, still lacking:
 [(1, 2, 3), (4, 5, 6), (7, 8, 9)]
Betterest Grouper can handle this:
 [(1, 2, 3), (4, 5, 6), (7, 8, 9), (10, None, None)]


### Brute Force seems rude
---

In [38]:
# Interview question:  You have three $20 bills, five $10 bills,
# two $5 bills, and five $1 bills. How many ways can you make change
# for a $100 bill?

# Basic brute force approach.  Iterate through all the possible
# combinations of bills in your wallet and check if they add up
# to $100
bills = [20, 20, 20, 10, 10, 10, 10, 10, 5, 5, 1, 1, 1, 1, 1]
# A choice of k things from a set of n things is called a combination
# itertools has a function for that: itertools.combinations(set, n)

# Let's see all the different combinations of 3 bills we could make
list(it.combinations(bills, 3))

# Nifty but not really helpful, yet. We can add an incrementor for
# bumping up the number of bills in our combination and a check to
# see if it makes 100 to validate our combinations against the 
# original constraint
makes_100=[]
# remember Python starts at 0 but we know 0 bills won't solve the problem
for n in range(1,len(bills)+1):
    for combo in it.combinations(bills, n):
        if sum(combo) == 100:
            makes_100.append(combo)
            
# We use set() to make sure our combos are unique
PP.pprint(set(makes_100))



{   (20, 20, 10, 10, 10, 10, 10, 5, 1, 1, 1, 1, 1),
    (20, 20, 10, 10, 10, 10, 10, 5, 5),
    (20, 20, 20, 10, 10, 10, 5, 1, 1, 1, 1, 1),
    (20, 20, 20, 10, 10, 10, 5, 5),
    (20, 20, 20, 10, 10, 10, 10)}
